In [ ]:
import glob
import pandas as pd
import scanpy as sc

import argparse
import sys
sys.path.append('../')

from arboreto.algo import grnboost2
from distributed import LocalCluster, Client

### ---------- ARGS

parser = argparse.ArgumentParser()
split_dir ='../data/norman2019/splits' ## Generate dataloader+splits first using GEARS dataloader
split_id =1
n_workers =50
adata_path=None
method='grnboost'

#args = parser.parse_args()
if adata_path is None:
    adata = sc.read_h5ad(split_dir.split('splits')[0]+'perturb_processed.h5ad')
else:
    adata = sc.read_h5ad(adata_path)

### ---------- 

def get_split_adata(adata, split_dir, split_id):
    split_files = [f for f in glob.glob(split_dir + '/*') if 'subgroup' not in f]
    split_fname = [f for f in split_files if 'simulation_'+str(split_id) in f][0]
    split_dict = pd.read_pickle(split_fname)
    
    return adata[adata.obs['condition'].isin(split_dict['train'])]
    

def get_grnboost_GRN(split_adata, n_workers):
    TF_names = pd.read_csv('../data/TF_names_v_1.01.txt', delimiter='\t', header=None)
    TF_names = TF_names[0].values

    all_conds = [c.split('+') for c in adata.obs['condition'].values ]
    all_conds = [item for sublist in all_conds for item in sublist]
    all_conds = set(all_conds)

    aug_TF_names = list(TF_names) + list(all_conds)
    
    exp_matrix = split_adata.X.toarray()
    gene_names = split_adata.var['gene_name'].values
    
    local_cluster = LocalCluster(n_workers=n_workers, 
                             threads_per_worker=1)
    custom_client = Client(local_cluster)
    adjacencies = grnboost2(exp_matrix, tf_names=aug_TF_names, 
                        verbose=True, gene_names=gene_names,
                        client_or_address=custom_client)
    
    return adjacencies

def save_split_GRN(adata, split_dir, split_id, n_workers):
    
    split_adata = get_split_adata(adata, split_dir, split_id)
    
    if method == 'grnboost':
        adjacencies = get_grnboost_GRN(split_adata, n_workers)
    
    adjacencies.to_csv(split_dir +'/adjacencies_'+str(split_id)+'.csv')
    

save_split_GRN(adata, split_dir, 
               split_id, n_workers)

preparing dask client


distributed.diskutils - INFO - Found stale lock file and directory '/dfs/user/yhr/gears2/linear/dask-worker-space/worker-r8j_58qb', purging
distributed.diskutils - INFO - Found stale lock file and directory '/dfs/user/yhr/gears2/linear/dask-worker-space/worker-kvjumb02', purging
distributed.diskutils - INFO - Found stale lock file and directory '/dfs/user/yhr/gears2/linear/dask-worker-space/worker-b1wfllp8', purging
distributed.diskutils - INFO - Found stale lock file and directory '/dfs/user/yhr/gears2/linear/dask-worker-space/worker-7m9ut7y7', purging
distributed.diskutils - INFO - Found stale lock file and directory '/dfs/user/yhr/gears2/linear/dask-worker-space/worker-inoddln_', purging
distributed.diskutils - INFO - Found stale lock file and directory '/dfs/user/yhr/gears2/linear/dask-worker-space/worker-w_lyacz3', purging
distributed.diskutils - INFO - Found stale lock file and directory '/dfs/user/yhr/gears2/linear/dask-worker-space/worker-cayfzy2e', purging
distributed.diskutil

parsing input
creating dask graph
10 partitions
computing dask graph
shutting down client and local cluster


Process Dask Worker process (from Nanny):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/dfs/user/yhr/scenic_env/lib/python3.7/site-packages/distributed/process.py", line 175, in _run
    target(*args, **kwargs)
  File "/dfs/user/yhr/scenic_env/lib/python3.7/site-packages/distributed/nanny.py", line 933, in _run
    loop.run_sync(do_stop)
  File "/dfs/user/yhr/scenic_env/lib/python3.7/site-packages/tornado/ioloop.py", line 523, in run_sync
    self.start()
  File "/dfs/user/yhr/scenic_env/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.asyncio_loop.run_forever()
  File "/usr/local/lib/python3.7/asyncio/base_events.py", line 534, in run_forever
    self._run_once()
  File "/usr/local/lib/python3.7/asyncio/base_events.py", line 1771